# Simulação de Reservatórios de Petróleo – 2021/2

## Trabalho 2

**Grupo:** Luan Rodrigues e Matheus Harduim.

**Professor:** Grazione de Souza.

**Instituição:** UERJ – IPRJ – DMC

### Desenvolvimento dos códigos e comentários.

Importando as bibliotecas do python que serão utilizadas